In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [5]:

import sys
sys.path.append('./')

import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np

import pickle
import faiss                     # make faiss available
import faiss.contrib.torch_utils
from numba import njit, prange
from numba import types
from numba.extending import overload

from shortest_path_onl import ShortestObj
from tqdm import tqdm

import os

In [7]:
with open('/home/ubuntu/fewshot3d_ws/geoformer/data/scannetv2/geoformer_scene_info_train.pkl', 'rb') as f:
    geoformer_scene_info_train = pickle.load(f)

with open('/home/ubuntu/fewshot3d_ws/geoformer/data/scannetv2/geoformer_knn_train.pkl', 'rb') as f:
    geoformer_knn_train = pickle.load(f)

In [8]:
shortestObj = ShortestObj()


In [16]:
for scene_name in tqdm(list(geoformer_scene_info_train.keys())):
    scene_dict = geoformer_scene_info_train[scene_name]
    knn = geoformer_knn_train[scene_name]
    locs_float_ = torch.from_numpy(scene_dict['locs_float_'])
    pre_enc_inds = torch.from_numpy(scene_dict['pre_enc_inds_arr'][0]).long()
    query_inds = pre_enc_inds[:128] # first 128 indices

    distances_arr = knn['distances_arr']
    indices_arr = knn['indices_arr']
    geo_dist = shortestObj.shortest_path(locs_float_, query_inds, distances_arr, indices_arr)

    saved_file = os.path.join('/home/ubuntu/fewshot3d_ws/geoformer/data/scannetv2/geoformer_geodist', f'geo_dist_{scene_name}')
    np.save(saved_file, geo_dist)


100%|██████████| 1192/1192 [2:39:14<00:00,  8.02s/it] 
